In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from pathlib import Path
import math
import pandas as pd
import altair as alt
# alt.renderers.enable('notebook')
from collections import Counter
import numpy as np
import csv
import random

random.seed(1337)

from news_utils import plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
BASE_PATH = Path('/mnt/data/datasets/ydata-ynacc-v1_0')
PATH = BASE_PATH/'ydata-ynacc-v1_0_expert_annotations.tsv'
PATH_TRAIN = BASE_PATH/'ydata-ynacc-v1_0_train-ids.txt'
PATH_VAL = BASE_PATH/'ydata-ynacc-v1_0_dev-ids.txt'
PATH_TEST = BASE_PATH/'ydata-ynacc-v1_0_test-ids.txt'
PATH_ARTICLES = Path('/mnt/data/group07/johannes/ynacc_proc/articles/articles_fixed_4.csv')

In [8]:
df = pd.read_table(PATH)

In [9]:
df

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
0,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
1,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,NaN,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Broadcast message / general audience,Not persuasive
2,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive
3,135929,0,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,fixyWJivQjEQtPLLVXsu,1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,1462203719,18.0,3.0,"I am frankly quite SICK of the phrase ""shoved ...",NaN,Not constructive,Agreement throughout,Off-topic/digression,negative,Mean,NaN,Off-topic with article,Broadcast message / general audience,Persuasive
4,135929,1,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,_TDnK715vO5y0OzZz_n4,00002I000000000000000000000000-7ef2ac58-bd84-4...,1462204643,7.0,2.0,"Ya, I always wonder why the conservatives are ...",1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,Not constructive,Agreement throughout,Off-topic/digression,neutral,Sarcastic,Agreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
5,135929,2,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,JzxshZNmXmdh_2kJN2E9,00003b000000000000000000000000-4a5ccbc3-4a6f-4...,1462239616,1.0,NaN,Great comment!,1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,Not constructive,Agreement throughout,Off-topic/digression,positive,NaN,Agreement with commenter,Off-topic with article,Broadcast message / general audience,Not persuasive
6,127970,0,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,mL_LzSXwMfJ21PfZ~yYv,1461069703997-4fe466e4-91e3-4095-a64b-5d89e8a3...,1461069703,10.0,5.0,The result is a system where being poor become...,NaN,Not constructive,Continual disagreement,Argumentative (back and forth),neutral,NaN,NaN,NaN,Broadcast message / general audience,Persuasive
7,127970,2,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,lc5tljQ08YooL7Ep3wT3,00003n000000000000000000000000-f1c6653c-a877-4...,1461070200,NaN,NaN,They are also places where you are supposed no...,1461069703997-4fe466e4-91e3-4095-a64b-5d89e8a3...,Not constructive,Continual disagreement,Argumentative (back and forth),neutral,Sarcastic,Disagreement with commenter,NaN,Reply to a specific commenter,Persuasive
8,127970,1,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,fuqaPUBVOAnc0o1gGEIg,00002b000000000000000000000000-498bb84f-8c31-4...,1461069989,1.0,1.0,"Stop trying to make sense, it only confuses pe...",1461069703997-4fe466e4-91e3-4095-a64b-5d89e8a3...,Not constructive,Continual disagreement,Argumentative (back and

Why are there so many comment? Because there are multiple evaluations for the same comment.

In [10]:
print(len(df.text.unique()))
print(len(df.commentid.unique()))

9146
9160


In [11]:
def maj(arr):
    c = Counter(arr)
    most = c.most_common(2)
    if len(most) > 1 and most[0][1] == most[1][1]:
        return -1
    return most[0][0]

def clear_winner(arr):
    c = Counter(arr)
    most = c.most_common(2)
    if len(most) > 1:
        return -1
    return most[0][0]

# NB: random seed of 1337
def maj_and_random(arr):
    c = Counter(arr)
    most = c.most_common(2)
    # no clear decision, we have to choose randomly
    if len(most) > 1:
        all_items =  list(c.items())
        best = [it for it in all_items if it[1] == most[0][1]]
        return random.sample(best, 1)[0][0]
    return most[0][0]

# Assing proper classes

In [12]:
df_a = df
decision_fun = maj_and_random

col_dict = {
    'topic': ['off-topic', 'on-topic'],
    'audience': ['broadcast', 'reply'],
    'persuasive': ['not persuasive', 'persuasive'],
    'sentiment': ['negative', 'neutral', 'mixed', 'positive'],
    'agreement': ['no agreement', 'agreement'],
    'disagreement': ['no disagreement', 'disagreement'],
    'informative': ['not informative', 'informative'],
    'mean': ['not mean', 'mean'],
    'controversial': ['not controversial', 'controversial'],
}

def get_index(key, attr):
#     print(attr)
    return col_dict[key].index(attr)

def set_audience(x):
    val = x['intendedaudience']
    if not pd.isnull(val):
        if val == "Broadcast message / general audience":
            return get_index("audience", "broadcast")
        if val == "Reply to a specific commenter":
            return get_index("audience", "reply")
    return np.NaN

df_a['claudience'] = df_a.apply(set_audience, axis=1)
df_a['clpersuasive'] = df_a.apply(lambda x: get_index('persuasive', 'persuasive') if isinstance(x['persuasiveness'], str) and x['persuasiveness'] == 'Persuasive' else get_index('persuasive', 'not persuasive'), axis=1)
df_a['clsentiment'] = df_a.apply(lambda x: get_index('sentiment', x['sentiment']) if isinstance(x['sentiment'], str) else np.NaN, axis=1)
df_a['clagreement'] = df_a.apply(lambda x: get_index('agreement', 'agreement') if x['commentagreement'] == "Agreement with commenter" else get_index('agreement', 'no agreement'), axis=1)
df_a['cldisagreement'] = df_a.apply(lambda x: get_index('disagreement', 'disagreement') if x['commentagreement'] == "Disagreement with commenter" else get_index('disagreement', 'no disagreement'), axis=1)
df_a['clinformative'] = df_a.apply(lambda x: get_index('informative', 'informative') if not pd.isnull(x['tone']) and "Informative" in x['tone'] else get_index('informative', 'not informative'), axis=1)
df_a['clmean'] = df_a.apply(lambda x: get_index('mean', 'mean') if not pd.isnull(x['tone']) and "Mean" in x['tone'] else get_index('mean', 'not mean'), axis=1)
df_a['clcontroversial'] = df_a.apply(lambda x: get_index('controversial', 'controversial') if not pd.isnull(x['tone']) and "Controversial" in x['tone'] else get_index('controversial', 'not controversial'), axis=1)
df_a['cltopic'] = df_a.apply(lambda x: get_index('topic','off-topic') if isinstance(x['topic'], str) and 'Off-topic with article' in x['topic'] else 
                           get_index('topic','on-topic'), axis=1)

print(df_a.shape)
# df_a = df_a.dropna(subset=[col_name])

for x in col_dict.keys():
    col_name = 'cl' + x
    res_group = df_a.groupby('commentid').agg({col_name: decision_fun}).reset_index()
    df_a = res_group.merge(df_a, on='commentid', how='left', suffixes=('', '_old_' + x)).reset_index()
    if 'level_0' in df_a.columns.tolist():
        df_a = df_a.drop(['level_0'], axis=1)

# remove columns added through merges
for x in df_a.columns.tolist():
    if '_old_' in x:
        df_a = df_a.drop([x], axis=1)

subset = ['commentid'] + ['cl' + x for x in col_dict.keys()]
df_a = df_a.drop_duplicates(subset=subset) # make sure it's really the same
df_a = df_a.reset_index(drop=True).reset_index(drop=True).reset_index(drop=True)
df_a['url'] = df_a.apply(lambda x: x['url'] if x['url'] != 'noURL' else np.NaN, axis=1)

# ensure we didn't destroy anthing
assert df_a.shape[0] == len(df.commentid.unique())

(23383, 29)


In [13]:
del df

In [14]:
df_a['clpersuasive'].value_counts()

0    7559
1    1601
Name: clpersuasive, dtype: int64

In [15]:
df_a['claudience'].value_counts()

1.0    6415
0.0    2679
Name: claudience, dtype: int64

In [16]:
df_a['clagreement'].value_counts()

0    8254
1     906
Name: clagreement, dtype: int64

In [17]:
df_a['cldisagreement'].value_counts()

0    5070
1    4090
Name: cldisagreement, dtype: int64

In [18]:
def write_table_for_ggplot(df, col, colname, idx):
    vc = df[col].value_counts()
    return ';'.join([str(idx), colname, str(vc[1]), 'yes']) + '\n' + ';'.join([str(idx + 1), colname, str(vc[0]), 'no'])

In [19]:
write_table_for_ggplot(df_a, 'cltopic', 'topic', 0)

'0;topic;6244;yes\n1;topic;2916;no'

In [97]:
all_cols = [['clpersuasive', 'Persuasiveness'],['claudience', 'Audience'], ['clagreement', 'Agreement'],
            ['cldisagreement', 'Disagreement'], ['clinformative', 'Informative'], ['clmean', 'Mean'],
            ['clcontroversial', 'Controversial'], ['cltopic', 'Off-Topic']
           ]

In [98]:
res = ''
for idx, x in enumerate(all_cols):
    print(write_table_for_ggplot(df_a, x[0], x[1], idx * 2))

0;Persuasiveness;1601;yes
1;Persuasiveness;7559;no
2;Audience;6415;yes
3;Audience;2679;no
4;Agreement;906;yes
5;Agreement;8254;no
6;Disagreement;4090;yes
7;Disagreement;5070;no
8;Informative;1534;yes
9;Informative;7626;no
10;Mean;1982;yes
11;Mean;7178;no
12;Controversial;3501;yes
13;Controversial;5659;no
14;Off-Topic;6244;yes
15;Off-Topic;2916;no


In [20]:
df_a['clsentiment'].value_counts()

0.0    5020
1.0    2311
2.0    1146
3.0     591
Name: clsentiment, dtype: int64

In [21]:
df_topic = df_a[['commentid', 'cltopic']]

In [22]:
df_topic

,commentid,cltopic
0,00002I000000000000000000000000-0f5bb163-9dbe-4...,0
1,00002I000000000000000000000000-4530d558-01ac-4...,1
2,00002I000000000000000000000000-62766fc8-bef0-4...,1
3,00002I000000000000000000000000-76055cf7-5fd1-4...,1
4,00002I000000000000000000000000-7ef2ac58-bd84-4...,0
5,00002I000000000000000000000000-9ccf6991-39e5-4...,0
6,00002I000000000000000000000000-b35e6d4c-a6b2-4...,1
7,00002I000000000000000000000000-d53278ae-be75-4...,0
8,00002S000000000000000000000000-1c86c54a-6711-4...,1
9,00002S000000000000000000000000-cf48a0e2-5130-4...,1


In [23]:
# there was a problem with the topic column, it was not probably set, so we have to go over all proceses arrays and fix it manually
fix_fn = 'train_proc'
fix_fn = 'train_proc_with_ner'
fix_fn = 'val_proc'
fix_fn = 'val_proc_with_ner'
fix_fn = 'test'


fix_df = pd.read_csv(Path('/mnt/data/group07/johannes/ynacc_proc/replicate/split/' + fix_fn + '.csv'))

fix_df = fix_df.drop(columns=['cltopic'])

fix_df = fix_df.merge(df_topic)
fix_df.to_csv(Path('/mnt/data/group07/johannes/ynacc_proc/replicate/split/' + fix_fn + '_fixed.csv'))

In [51]:
fix_df[['cltopic', 'text_proc']]

,cltopic,text_proc
0,0,Yes..because too many houses in xxorg look lik...
1,1,"These things happen , Every job has its dangers."
2,1,Sad to hear such a bad thing. Very dangerous j...
3,1,THANK YOU!!!!!!!!!!!!!!!! So annoying.. journa...
4,1,lol i thought it's my computer
5,1,It's on the xxorg article you dolt.
6,1,You have to click on the link to go to Cosmo. ...
7,1,Is it just my computer or is there not a singl...
8,0,why is she sick --American laws are sick --16 ...
9,1,"Still, the woman is the legal adult and is a m..."


In [12]:
# df_a.groupby(['sdid', 'constructiveclass']).count()

,,commentid,clcontroversial,clmean,clinformative,cldisagreement,clagreement,clsentiment,clpersuasive,claudience,index,...,text,parentid,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
sdid,constructiveclass,,,,,,,,,,,,,,,,,,,,,
27,Constructive,7,7,7,7,7,7,7,7,7,7,...,7,6,7,7,7,6,6,4,7,7
44,Constructive,7,7,7,7,7,7,7,7,7,7,...,7,6,7,7,7,6,6,3,7,7
80,Constructive,4,4,4,4,4,4,4,4,4,4,...,4,3,4,4,4,4,3,3,4,4
219,Not constructive,5,5,5,5,5,5,5,5,5,5,...,5,4,5,5,5,5,3,1,5,5
269,Constructive,3,3,3,3,3,3,3,3,3,3,...,3,2,3,3,3,3,2,0,3,3
288,Not constructive,5,5,5,5,5,5,5,5,5,5,...,5,4,5,5,5,5,4,5,5,5
442,Constructive,7,7,7,7,7,7,7,7,7,7,...,7,6,7,7,7,3,6,7,7,7
523,Not constructive,5,5,5,5,5,5,5,5,5,5,...,5,4,5,5,5,2,2,5,5,5
729,Constructive,3,3,3,3,3,3,3,3,3,3,...,3,2,3,3,3,3,2,0,2,3


# Combine with Articles

In [22]:
df_art = pd.read_csv(PATH_ARTICLES)
df_art = df_art.rename(index=str, columns={"text": "article_text"})

In [23]:
df_art

,Unnamed: 0,url,article_text,title,publish_date
0,1,http://www.cosmopolitan.com/lifestyle/news/a56...,A Disneyland Paris employee was found dead ins...,Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48
1,2,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00
2,3,http://www.insideedition.com/headlines/15888-e...,Headlines Teacher Allegedly Had Sex With Stude...,Teacher Allegedly Had Sex With Student On Nigh...,NaN
3,4,http://www.ozy.com/good-sht/the-dirtiest-socce...,Because never does a man stand so tall as when...,The Dirtiest Soccer Player in the Known Universe,NaN
4,5,http://www.womansday.com/health-fitness/a54485...,"Christina Phillips, who was profiled on TLC's ...",What Happened After This Woman Lost 537 Pounds...,2016-04-11 02:54:32
5,6,http://www.cosmopolitan.com/style-beauty/fashi...,"Old Navy\n\nOn Friday, Old Navy posted an ad o...",Old Navy Featured an Interracial Family in an ...,2016-05-02 02:03:00
6,7,http://www.vibe.com/2016/04/texas-cop-caught-b...,"Just days ago, video footage of a Texas cop bo...",Texas Cop Body-Slams 12-Year-Old Girl,2016-04-11 10:27:58+00:00
7,8,http://hellogiggles.com/trans-woman-selfie-bat...,Sarah McBride peed the other day. But it wasn’...,This trans woman just posted a very important ...,2016-04-23 08:46:16-07:00
8,9,http://mashable.com/2016/04/16/waitress-tip-bi...,A waitress from North Carolina claims she was ...,Gay North Carolina waitress receives bible ver...,2016-04-16 00:00:00
9,10,http://mashable.com/2016/04/08/capybaras-hide-...,What's cuter than one capybara hiding from the...,People are losing their cool over this pic of ...,2016-04-08 00:00:00


In [24]:
df_a.shape

(9160, 30)

In [25]:
df_b = pd.merge(df_art, df_a, how='right', on='url')

In [26]:
df_art[df_art.duplicated(subset=['url'], keep=False)]

,Unnamed: 0,url,article_text,title,publish_date


In [27]:
len(df_art['url'].unique())

670

In [28]:
df_b.shape

(9160, 34)

In [29]:
df_b[df_b['url'].isnull()]

,Unnamed: 0,url,article_text,title,publish_date,commentid,clcontroversial,clmean,clinformative,cldisagreement,...,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
8983,NaN,NaN,NaN,NaN,NaN,00002b000000000000000000000000-1034cd3d-f3ea-4...,1,0,0,1,...,1470849970031-a71df58d-eddd-4e6c-9960-de151adc...,Constructive,Continual disagreement,"Argumentative (back and forth),Flamewar (insul...",negative,Controversial,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
8984,NaN,NaN,NaN,NaN,NaN,00002b000000000000000000000000-2336ec2a-d408-4...,1,0,1,0,...,1470460286738-25b5cde7-36e7-4d3c-9d4f-70cf3037...,Constructive,Initial agreement leading to disagreement,"Argumentative (back and forth),Flamewar (insul...",neutral,"Informative,Controversial",Disagreement with commenter,NaN,Reply to a specific commenter,Persuasive
8985,NaN,NaN,NaN,NaN,NaN,00002b000000000000000000000000-409e8926-5966-4...,1,0,1,0,...,1471217041167-2b91ce17-0739-41e8-8a41-f11f22c1...,Constructive,Agreement throughout,"Argumentative (back and forth),Positive/respec...",mixed,"Informative,Controversial",Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
8986,NaN,NaN,NaN,NaN,NaN,00002b000000000000000000000000-5f80f211-362a-4...,1,0,0,0,...,1469745136053-66ed8012-7501-4019-91e3-cf11ee4a...,Not constructive,NaN,Off-topic/digression,negative,Controversial,Adjunct opinion,Off-topic with article,Reply to a specific commenter,NaN
8987,NaN,NaN,NaN,NaN,NaN,00002b000000000000000000000000-703ef2ee-04b8-4...,1,0,1,0,...,1471123337593-e106540b-45cb-40b2-8a0e-ca0db9f9...,Not constructive,Continual disagreement,"Argumentative (back and forth),Snarky/humorous...",neutral,"Informative,Controversial",Adjunct opinion,Off-topic with conversation,Broadcast message / general audience,Not persuasive
8988,NaN,NaN,NaN,NaN,NaN,00002g000000000000000000000000-0e7f1b2f-3a0f-4...,0,0,0,1,...,1471375862684-dc166994-85e6-40f0-ae0d-2114c160...,Not constructive,Continual disagreement,"Argumentative (back and forth),Snarky/humorous",negative,"Controversial,Mean",Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
8989,NaN,NaN,NaN,NaN,NaN,00002g000000000000000000000000-157e6c56-51db-4...,1,0,0,1,...,1469570957765-2165fe8a-625c-406f-931c-fb64d5c4...,Not constructive,NaN,"Argumentative (back and forth),Off-topic/digre...",negative,Controversial,Adjunct opinion,NaN,Reply to a specific commenter,Persuasive
8990,NaN,NaN,NaN,NaN,NaN,00002g000000000000000000000000-323da914-187c-4...,1,0,0,1,...,1471121127802-87672757-a504-4492-b0cd-ee8572d6...,Not constructive,Continual disagreement,Argumentative (back and forth),mixed,Controversial,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
8991,NaN,NaN,NaN,NaN,NaN,00002g000000000000000000000000-43c5dc0b-f40e-4...,0,0,0,1,...,1469677361096-4165cb38-fe60-46ed-853b-c66915e7...,Not constructive,NaN,NaN,neutral,Sarcastic,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
8992,NaN,NaN,NaN,NaN,NaN,00002g000000000000000000000000-5792c6e6-53a2-4...,1,0,0,1,...,1469582617167-8769bdc1-7b09-444a-b8f0-64b4c0b7...,Not constructive,Initial disagreement converging to agreement,Argumentative (back and forth),mixed,Controversial,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive


In [30]:
df_b = df_b.drop(['Unnamed: 0'], axis=1)

In [31]:
df_b

,url,article_text,title,publish_date,commentid,clcontroversial,clmean,clinformative,cldisagreement,clagreement,...,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
0,http://www.cosmopolitan.com/lifestyle/news/a56...,A Disneyland Paris employee was found dead ins...,Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48,00002n000000000000000000000000-1c30b878-b717-4...,0,0,1,0,0,...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive
1,http://www.cosmopolitan.com/lifestyle/news/a56...,A Disneyland Paris employee was found dead ins...,Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48,00003n000000000000000000000000-ed2ae6d0-32ac-4...,0,0,0,0,0,...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
2,http://www.cosmopolitan.com/lifestyle/news/a56...,A Disneyland Paris employee was found dead ins...,Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,0,0,0,0,0,...,NaN,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Broadcast message / general audience,Not persuasive
3,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00,00002g000000000000000000000000-c08bc8b6-ecd2-4...,0,0,0,0,1,...,1459864307079-aeebbc73-f562-4540-8576-cdcebe13...,Not constructive,NaN,NaN,neutral,NaN,Adjunct opinion,NaN,Reply to a specific commenter,Not persuasive
4,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00,00003g000000000000000000000000-5f0375ec-8977-4...,0,0,0,0,1,...,1459864307079-aeebbc73-f562-4540-8576-cdcebe13...,Not constructive,NaN,NaN,neutral,NaN,Agreement with commenter,NaN,Reply to a specific commenter,Not persuasive
5,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00,00004n000000000000000000000000-c44d1de5-af2f-4...,0,1,0,1,0,...,1459864307079-aeebbc73-f562-4540-8576-cdcebe13...,Not constructive,NaN,NaN,negative,Mean,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive
6,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00,00005n000000000000000000000000-d0c8ecc0-0216-4...,0,0,0,0,0,...,1459864307079-aeebbc73-f562-4540-8576-cdcebe13...,Not constructive,NaN,NaN,neutral,NaN,Adjunct opinion,NaN,Reply to a specific commenter,Not persuasive
7,http://www.cosmopolitan.com/health-fitness/new...,Having documented her baby bump in mirror self...,See Fitness Model Chontel Duncan's Post-Baby B...,2016-04-01 03:17:00,1459864307079-aeebbc73-f562-4540-8576-cdcebe13...,0,0,0,0,0,...,NaN,Not constructive,NaN,NaN,neutral,NaN,NaN,NaN,Broadcast message / general audience,Not persuasive
8,http://www.insideedition.com/headlines/15888-e...,Headlines Teacher Allegedly Had Sex With Stude...,Teacher Allegedly Had Sex With Student On Nigh...,NaN,00002b000000000000000000000000-a22824ca-7630-4...,0,0,0,1,0,...,1460818837098-c6d3ff5e-f271-4f00-8fc5-935aa542...,Not constructive,Continual disagreement,"Snarky/humorous,Flamewar (insulting)",negative,"Controversial,Mean",Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
9,http://www.insideedition.com/headlines/15888-e...,Headlines Teacher Allegedly Had Sex With Stude...,Teacher Allegedly Had Sex With Student On Nigh...,NaN,00002b00000000000

# Use provided Splits

In [32]:
OUT_PATH = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/')
Path(OUT_PATH).mkdir(parents=True, exist_ok=True)

In [33]:
train = pd.read_csv(PATH_TRAIN, header=None)
val = pd.read_csv(PATH_VAL, header=None)
test = pd.read_csv(PATH_TEST, header=None)

In [34]:
len(train[0].unique())

2130

In [35]:
val.shape

(97, 1)

In [36]:
test.shape

(100, 1)

In [37]:
len(df_b['sdid'].unique())

1400

In [38]:
df_tr = df_b[df_b['sdid'].isin(list(train[0]))]
df_va = df_b[df_b['sdid'].isin(list(val[0]))]
df_te = df_b[df_b['sdid'].isin(list(test[0]))]

In [39]:
df_tr.shape[0] + df_va.shape[0] + df_te.shape[0]

9046

Why it does not match? The provided IDs are also for the turk annotations. They don't have comment label annotations so we can't use them. Still not sure why there are some comments missing (over 100). Probably not in the provided split

In [40]:
def save(df, kind):
    print(len(df))
    df.to_csv((OUT_PATH/kind).with_suffix('.csv'), header=True, index=False, quoting=csv.QUOTE_ALL)

In [41]:
save(df_tr, 'train')
save(df_va, 'val')
save(df_te, 'test')

7910
583
553


### Leftover: Save URL to crawl

In [17]:
# save urls
# pd.DataFrame(df_a['url'].unique()).to_csv((OUT_PATH/'urls').with_suffix('.csv'), header=False, index=False)

In [1]:
7910 / 9046

0.8744196329869556

In [2]:
583 / 9046

0.06444837497236347

In [3]:
553 / 9046

0.06113199204068096